# Análisis y limpieza de datos 

# Parte 1
¿Cómo ha cambiado la proporción entre películas y series a lo
largo de los años?

# Parte 2
¿Cómo evolucionó la cantidad de estrenos a lo largo del tiempo
para cada tipo de contenido?

# Parte 3
¿Cuáles son los países con más producciones? ¿Se destacan
más por películas o series?

# Parte 4
¿Qué tipo de contenido es más común para cada rating?

# Parte 5
¿Qué países producen más contenido para audiencias adultas e
infantiles? ¿Se les ocurre otra separación además de estas dos?

# Parte 6
¿Se identifica alguna estacionalidad en los estrenos según la
categoría (listed in)? Qué meses concentran más lanzamientos?

# Parte 7
¿Qué directores tienen más títulos? ¿Se concentran en algún tipo de contenido o en una audiencia específica?

# Parte 8
¿Cuáles son los actores más populares?


# Parte 9
¿Cuál es la distribución en duración en series y películas?

# Parte 10
¿Hay palabras que se utilicen más que otras en títulos y descripciones?